In [1]:
using QEDFeynmanDiagrams
using QEDcore
using QEDprocesses
using ComputableDAGs

In [2]:
n = 5
proc = ScatteringProcess(
    (Electron(), ntuple(_ -> Photon(), n)...),  # incoming particles
    (Electron(), Photon()),                     # outgoing particles
    (AllSpin(), ntuple(_ -> PolX(), n)...),     # incoming particle spin/pols
    (AllSpin(), AllPol())                       # outgoing particle spin/pols
) # compton: k^n e -> ke, with all spin/pol combinations except incoming photons
psp = PhaseSpacePoint(
    proc,
    PerturbativeQED(),
    PhasespaceDefinition(SphericalCoordinateSystem(), ElectronRestFrame()),
    tuple((rand(SFourMomentum) for _ in 1:number_incoming_particles(proc))...),
    tuple((rand(SFourMomentum) for _ in 1:number_outgoing_particles(proc))...)
)
println("Diagrams for the process: $(length(feynman_diagrams(proc)))")
graph = generate_DAG(proc)

Diagrams for the process: 720


Graph:
  Nodes: Total: 2605, QEDFeynmanDiagrams.ComputeTask_Triple: 480, QEDFeynmanDiagrams.ComputeTask_BaseState: 11, 
         DataTask: 1339, QEDFeynmanDiagrams.ComputeTask_Propagator: 62, QEDFeynmanDiagrams.ComputeTask_CollectPairs: 168, 
         QEDFeynmanDiagrams.ComputeTask_CollectTriples: 8, QEDFeynmanDiagrams.ComputeTask_SpinPolCumulation: 1, QEDFeynmanDiagrams.ComputeTask_PropagatePairs: 168, 
         QEDFeynmanDiagrams.ComputeTask_Pair: 368
  Edges: 4707
  Total Compute Effort: 0.0
  Total Data Transfer: 0.0
  Total Compute Intensity: 0.0


In [3]:
# this is necessary when using RuntimeGeneratedFunctions
using RuntimeGeneratedFunctions
RuntimeGeneratedFunctions.init(@__MODULE__)

func = get_compute_function(graph, proc, cpu_st(), @__MODULE__)
func(psp)

0.012329630899041492

In [4]:
using BenchmarkTools
@benchmark func($psp)

BenchmarkTools.Trial: 10000 samples with 1 evaluation.
 Range (min … max):  82.900 μs …  30.548 ms  ┊ GC (min … max): 0.00% … 99.42%
 Time  (median):     88.966 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   98.580 μs ± 306.680 μs  ┊ GC (mean ± σ):  4.94% ±  5.00%

  ▁▁▆█▆▅▄▄▃▃▃▂▂▂▁                     ▁▁                       ▂
  ███████████████████▇▆▆▅▆▆▆▆▅▃▄▁▄▅▄▆█████▇▆▆▅▆▅▃▄▅▅▄▄▁▄▄▃▄▅▅▆ █
  82.9 μs       Histogram: log(frequency) by time       176 μs <

 Memory estimate: 184.94 KiB, allocs estimate: 1378.